Imports

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score
os = SMOTE(random_state=0)

from google.colab import drive

drive.mount('/content/drive')
jobs = pd.read_csv('/content/drive/My Drive/Colab Notebooks/HR_jobs_predict.csv')
print(jobs)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
       satisfaction_level  last_evaluation  number_project  \
0                    0.38             0.53               2   
1                    0.80             0.86               5   
2                    0.11             0.88               7   
3                    0.72             0.87               5   
4                    0.37             0.52               2   
...                   ...              ...             ...   
14994                0.40             0.57               2   
14995                0.37             0.48               2   
14996                0.37             0.53               2   
14997                0.11             0.96               6   
14998                0.37             0.52               2   

       average_montly_hours  time_spend_company  Work_accident  left  \
0                       157                   3       

Question 1

---

Describe the Dataset

Problem Statement

Let's discover which of the variables had a clear and direct impact on employee retention ( meaning what kept or made employees left )


Data Descrpition

The data itself shows employee information such as: Satisfaction, Amount of Projects, Avg Monthly Hours, Time at Company... and many more. All things that can determine the longevity of an employees stay at an company.  

Target Variable

I am going to go with the left column, which will show us everyone that decided to stay and who decided to leave in the end.

Prediction Goal

We will be able to predict which column the company needs to work towards to retain a higher keep of their employees at the company.

Question 2

---

Data Cleaning

Drop Unneccesary Data

In [ ]:
jobs = jobs.drop(columns=['Department'])
print(jobs)

Address Missing Data

In [ ]:
jobs.columns
jobs.dropna()

Remove Duplicates

N/A, data was already semi cleaned, there was not much to do to it

Check Inconsistencies within categorical values

N/A, data was already semi cleaned, there was not much to do to it

Turn categorical values into numerical values

In [ ]:
jobs['salary'] = pd.factorize(jobs['salary'])[0]
print(jobs)

Create Boxplots / Histograms to check and get rid of outliers

In [ ]:
df_jobs = jobs.rename(columns={'left': 'target'})
df_jobs.info()

In [ ]:
from re import sub
target_value = 'target'
nums_list = ['number_project','average_montly_hours','time_spend_company','salary']
fig=plt.figure(figsize=(10,8))

for i in range(len(nums_list)):
  column=nums_list[i]
  sub=fig.add_subplot(2,2,i+1)
  sns.boxplot(x=df_jobs['target'],y=column,data=jobs, showfliers=False)

Question 3


---

Feature Engineering

ANOVA

In [ ]:
x = df_jobs.loc[:, ['number_project','average_montly_hours','time_spend_company','salary']]
y = df_jobs.loc[:, 'target']

fs = SelectKBest(score_func=f_classif, k='all')
FeaturesS = fs.fit(x,y)
np.set_printoptions(suppress = True )
print(FeaturesS.scores_)
print(FeaturesS.pvalues_)


In [ ]:
print(df_jobs.corr())

CHI-SQUARED

In [ ]:
x = df_jobs.loc[:, ['number_project','average_montly_hours','time_spend_company','salary']]
y = df_jobs.loc[:, 'target']

chi = SelectKBest(score_func=chi2, k='all')
catFeatures = chi.fit(x, y)
print(catFeatures.scores_)
print(catFeatures.pvalues_)


Little to no varianvce

---

irrelevant variance


---

highly correlated variance

Question 4

---

Prediction Model

In [15]:
x = df_jobs.loc[:, ['number_project','average_montly_hours','time_spend_company','salary']]
y = df_jobs.loc[:, 'target']

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=0, stratify=y)
print(x_train.shape)

(10499, 4)


Logistic Prediction Model

In [16]:
oversampled_x,oversampled_y=os.fit_resample(x_train, y_train)
print(oversampled_x.shape)

(15998, 4)


In [22]:
regression = LogisticRegression(penalty='none', max_iter=2000)
regression.fit(oversampled_x, oversampled_y.values.ravel())

test_pred = regression.predict(x_test)
accuracy_score(y_test, test_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


0.6042222222222222